In [1]:
from google.colab import drive
drive.mount('/content/drive')
""

Mounted at /content/drive


''

In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/test/clean_data.xlsx')

In [14]:
az_df = df[df['state'] == 'AZ']
unique_item_codes = az_df['item_code'].nunique()
print(f"Number of unique item codes in Arizona: {unique_item_codes}")
print(f"Number of unique item codes: ({df['item_code'].nunique()})")


Number of unique item codes in Arizona: 745
Number of unique item codes: (1914)


In [15]:
store_1012 = df[df['store_number'] == 1012]
print(store_1012.info())

<class 'pandas.core.frame.DataFrame'>
Index: 282 entries, 37960 to 38241
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   state                         282 non-null    object 
 1   item_code                     282 non-null    int64  
 2   package_type                  282 non-null    object 
 3   sales_dollars_L52wk           282 non-null    float64
 4   points_of_distribution_L52wk  282 non-null    float64
 5   store_number                  282 non-null    int64  
 6   Retail                        282 non-null    float64
 7   L52W_in_Stock                 282 non-null    int64  
 8   Normalized_Sales_L52W         265 non-null    float64
 9   Sales_Bucket                  282 non-null    object 
 10  PriceZone                     282 non-null    object 
 11  Store_Size                    282 non-null    object 
 12  HH_Income>100K                282 non-null    float64
 13  Medi

In [16]:
import numpy as np
item_codes_not_in_1012 = df[~df['item_code'].isin(store_1012['item_code'])]
# print(item_codes_not_in_1012)
# Create a new DataFrame with unique rows for store 1012
new_rows = []
for _, row in item_codes_not_in_1012.iterrows():
    new_row = {
        'state': 'AZ',
        'store_number': 1012,
        'item_code': row['item_code'],
        'package_type': row['package_type'],
        'Retail': row['Retail'],
        'Sales_Bucket': 'Sales included',
        'PriceZone': store_1012['PriceZone'].iloc[0],
        'Store_Size': store_1012['Store_Size'].iloc[0],
        'HH_Income>100K': store_1012['HH_Income>100K'].iloc[0],
        'Median_HH_Income': store_1012['Median_HH_Income'].iloc[0],
        'Average_NetWorth': store_1012['Average_NetWorth'].iloc[0],
        'HaveBA': store_1012['HaveBA'].iloc[0],
        'Hispanic': store_1012['Hispanic'].iloc[0],
        'Asian': store_1012['Asian'].iloc[0],
        'AfricanAmerican' : store_1012['AfricanAmerican'].iloc[0],
        'PopulationAge_50-70' : store_1012['PopulationAge_50-70'].iloc[0],
        'TequilaUnder65' : row['TequilaUnder65'],
        'TequilaOver65' : row['TequilaOver65'],
        'tequila_category' : row['tequila_category'],
        'price_per_100_ml' : row['price_per_100_ml'],
        'normalized_households': store_1012['normalized_households'].iloc[0],
        'price_band': row['price_band'],
        'volatility_score': row['volatility_score'],
        'households_density': store_1012['households_density'].iloc[0],
        'celebrity_owned': row['celebrity_owned'],
        'Income_Category': store_1012['Income_Category'].iloc[0],
        'years_open': store_1012['years_open'].iloc[0],
        'search_anejo_tequila': store_1012['search_anejo_tequila'].iloc[0],
        'score_tequila_trends': store_1012['score_tequila_trends'].iloc[0],
        'search_reposado_tequila': store_1012['search_reposado_tequila'].iloc[0],
        'search_silver_tequila': store_1012['search_silver_tequila'].iloc[0],
        'search_cheap_tequila': store_1012['search_cheap_tequila'].iloc[0],
        'search_best_tequila': store_1012['search_best_tequila'].iloc[0]
    }

    for col in df.columns:
        if col not in new_row.keys():
            new_row[col] = np.nan

    new_rows.append(new_row)

new_df = pd.DataFrame(new_rows)


store_1012 = pd.concat([store_1012, new_df], ignore_index=True)


store_1012 = store_1012.drop_duplicates(subset=['item_code', 'store_number'])



print(store_1012.info())
store_1012.shape

<class 'pandas.core.frame.DataFrame'>
Index: 1914 entries, 0 to 45320
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   state                         1914 non-null   object 
 1   item_code                     1914 non-null   int64  
 2   package_type                  1914 non-null   object 
 3   sales_dollars_L52wk           282 non-null    float64
 4   points_of_distribution_L52wk  282 non-null    float64
 5   store_number                  1914 non-null   int64  
 6   Retail                        1914 non-null   float64
 7   L52W_in_Stock                 282 non-null    float64
 8   Normalized_Sales_L52W         265 non-null    float64
 9   Sales_Bucket                  1914 non-null   object 
 10  PriceZone                     1914 non-null   object 
 11  Store_Size                    1914 non-null   object 
 12  HH_Income>100K                1914 non-null   float64
 13  Median_

(1914, 38)

In [17]:
store_1012['item_code'].nunique()

1914

In [18]:
import pandas as pd
import numpy as np


store_1012['sales_adjusted'] = store_1012['item_code'].map(
    df[df['state'] == 'AZ'].groupby('item_code')['sales_dollars_L52wk'].mean()
) * store_1012['normalized_households']

store_1012['sales_dollars_L52wk'] = np.where(
    store_1012['sales_dollars_L52wk'].isna(),
    store_1012['sales_adjusted'],
    store_1012['sales_dollars_L52wk']
)


store_1012['sales_adjusted2'] = store_1012.set_index(['state', 'Retail']).index.map(
    df[df['state'] == 'AZ'].groupby(['state', 'Retail'])['sales_dollars_L52wk'].mean()
) * store_1012['normalized_households']

store_1012['sales_dollars_L52wk'] = np.where(
    store_1012['sales_dollars_L52wk'].isna(),
    store_1012['sales_adjusted2'],
    store_1012['sales_dollars_L52wk']
)


store_1012['sales_adjusted3'] = store_1012['Retail'].map(
    df[df['state'] == 'AZ'].groupby('Retail')['sales_dollars_L52wk'].mean()
) * store_1012['normalized_households']

store_1012['sales_dollars_L52wk'] = np.where(
    store_1012['sales_dollars_L52wk'].isna(),
    store_1012['sales_adjusted3'],
    store_1012['sales_dollars_L52wk']
)


store_1012['price_band'] = pd.cut(
    store_1012['Retail'],
    bins=[-np.inf, 20, 65, 150, np.inf],
    labels=[1, 2, 3, 4]
)

store_1012['sales_adjusted4'] = store_1012.set_index(['state', 'price_band']).index.map(
    df[df['state'] == 'AZ'].groupby(['state', 'price_band'])['sales_dollars_L52wk'].mean()
) * store_1012['normalized_households']

store_1012['sales_dollars_L52wk'] = np.where(
    store_1012['sales_dollars_L52wk'].isna(),
    store_1012['sales_adjusted4'],
    store_1012['sales_dollars_L52wk']
)


store_1012.drop(['sales_adjusted', 'sales_adjusted2', 'sales_adjusted3', 'sales_adjusted4'], axis=1, inplace=True)


print(store_1012['sales_dollars_L52wk'].describe())


count    1.914000e+03
mean     3.731136e+04
std      2.042166e+05
min      0.000000e+00
25%      1.064128e+03
50%      3.290935e+03
75%      1.191529e+04
max      3.286171e+06
Name: sales_dollars_L52wk, dtype: float64


In [19]:
import pandas as pd
import numpy as np

# Calculate the overall median for `points_of_distribution_L52wk` from the entire dataset
overall_median_pod = df['points_of_distribution_L52wk'].median(skipna=True)

# Step 1: Calculate `avg_pod` for each `item_code`
store_1012['avg_pod'] = store_1012['item_code'].map(
    df.groupby('item_code')['points_of_distribution_L52wk'].apply(
        lambda x: overall_median_pod if x.isna().all() else x.mean(skipna=True)
    )
)

# Step 2: Normalize `normalized_households` if it isn't already normalized
store_1012['normalized_households'] = store_1012.groupby('item_code')['normalized_households'].transform(
    lambda x: x / x.max() if x.max() > 0 else x
)

# Step 3: Calculate `pod_adjusted` using `avg_pod` and `normalized_households`
store_1012['pod_adjusted'] = store_1012['avg_pod'] * store_1012['normalized_households']

# Step 4: Replace missing values in `points_of_distribution_L52wk` with `pod_adjusted`
store_1012['points_of_distribution_L52wk'] = np.where(
    store_1012['points_of_distribution_L52wk'].isna(),
    store_1012['pod_adjusted'],
    store_1012['points_of_distribution_L52wk']
)

#Drop intermediate columns
store_1012.drop(['avg_pod', 'pod_adjusted'], axis=1, inplace=True)

# Output summary of the `points_of_distribution_L52wk` column
print(store_1012['points_of_distribution_L52wk'].describe())


count    1914.000000
mean      164.429242
std       272.083350
min         0.000000
25%        31.588738
50%       104.254843
75%       189.892000
max      4151.171091
Name: points_of_distribution_L52wk, dtype: float64


In [23]:
store_1012['sales_per_store'] = np.where(
    store_1012['points_of_distribution_L52wk'] == 0,  # Check if points_of_distribution_L52wk is 0
    0,  # If true, set sales_per_store to 0
    store_1012['sales_dollars_L52wk'] / store_1012['points_of_distribution_L52wk']  # Otherwise, calculate the ratio
)

In [24]:
store_1012['L52W_in_Stock'] = store_1012['L52W_in_Stock'].fillna(52)

In [25]:
print(store_1012.isna().sum())

state                              0
item_code                          0
package_type                       0
sales_dollars_L52wk                0
points_of_distribution_L52wk       0
store_number                       0
Retail                             0
L52W_in_Stock                      0
Normalized_Sales_L52W           1649
Sales_Bucket                       0
PriceZone                          0
Store_Size                         0
HH_Income>100K                     0
Median_HH_Income                   0
Average_NetWorth                   0
HaveBA                             0
Hispanic                           0
Asian                              0
AfricanAmerican                    0
PopulationAge_50-70                0
TequilaUnder65                     0
TequilaOver65                      0
tequila_category                   0
price_per_100_ml                   0
normalized_households              0
price_band                         0
volatility_score                   0
h

In [26]:
# Save the DataFrame to an Excel file
store_1012.to_excel('/content/drive/MyDrive/test/store_1012.xlsx', index=False)